In [ ]:
import pandas as pd
import numpy as np
import pprint as pprint

In [ ]:
# Restaurants
dfaccepts=pd.read_csv(filepath_or_buffer='Restaurant-data/chefmozaccepts.csv',encoding='ascii')
dfcuisine=pd.read_csv(filepath_or_buffer='Restaurant-data/chefmozcuisine.csv',encoding='utf8')
dfhours4=pd.read_csv(filepath_or_buffer='Restaurant-data/chefmozhours4.csv',encoding='utf8')
dfparkingdata2=pd.read_csv('Restaurant-data/chefmozparking.csv')
dfgeoplaces=pd.read_csv(filepath_or_buffer='Restaurant-data/geoplaces2.csv',encoding='latin-1')
# # consumers
dfusercuisine=pd.read_csv('Restaurant-data/usercuisine.csv')
dfuserpayment=pd.read_csv('Restaurant-data/userpayment.csv')
dfuserprofile=pd.read_csv('Restaurant-data/userprofile.csv')
# # user item rating
dfrating=pd.read_csv('Restaurant-data/rating_final.csv')


In [ ]:
# 3.1. Concatenate all cleaned data frames from ‘Restaurant’ category based on the common column, to create a large data frame and name it as Restaurants.
dfrestaurants=dfgeoplaces.merge(dfcuisine,how='left',on='placeID').merge(dfhours4,how='left',on='placeID').merge(dfaccepts,how='left',on='placeID').merge(dfparkingdata2,how='left',on='placeID')
dfrestaurants.shape

In [ ]:
# 3.2. Concatenate all cleaned data frames from ‘Customer’ categories based on the common column, to create a large data frame and name it as Customers.
dfcustomers=dfuserprofile.merge(dfuserpayment,on='userID',how='left').merge(dfusercuisine, on='userID' , how='left')
dfcustomers.shape

In [ ]:
# 3.3. Import the required package and create an engine for the sqlalchemy application:
import sqlalchemy as sqla
engine = sqla.create_engine('sqlite:///data_science.db',echo=False)


In [ ]:
# 3.4. Create three SQL tables from three data frames: ‘Restaurants’, ‘Customers’
# which are created above, and the database from ‘rate’ category. The required
# syntax for creating each table is as follows:

dfrestaurants.to_sql(name='restaurants', con=engine, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)
dfcustomers.to_sql(name='customers', con=engine, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)
dfrating.to_sql(name='rating', con=engine, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)
# engine.execute("SELECT * FROM restaurants").fetchall()

Assignment 3
In this assignment, you will provide answers for the questions from Assignment
2, through making SQL databases and SQL queries, using a Pandas module
known as ‘sqlalchemy’.


3.5. Answer all the questions from the previous assignment (Assignment 2), using
SQL queries; except for Question 2.2.8 which is not needed to be answered in
Assignment 3.

Use the following command to query the databases:
• pd.read_sql(query, db), where ‘query’ is the SQL query for answering the
question.

In [ ]:
# 2.1.3. What are the names of different restaurants in the state of ‘tamaulipas’?
# select where name is tamaulipas
dfgeoplaces.loc[:,['name']][(dfgeoplaces.state=='tamaulipas')]
engine.execute("SELECT distinct name FROM restaurants where state='tamaulipas'").fetchall()


In [ ]:
# 2.1.4. How many different customers used public transport for going to therestaurants?
engine.execute("select  count(userID) from (SELECT  distinct userID FROM customers where transport='public')").fetchall()


In [ ]:
# 2.1.5. What is the least popular payment method among customers?
engine.execute(" select upayment,MIN(paymentcount) from (select upayment,count(upayment)as paymentcount from ( SELECT distinct upayment,userID FROM customers) where upayment!='None' group by upayment)").fetchall()#=======================get the least


In [ ]:
# 2.1.6. How many (different) restaurants work until 19:00 in the evenings?
# len(dfhours4[dfhours4=='00:00-19:00'])
dfhours4.hours
# ===================================================
# pd.to_timedelta(dfhours4.hours)

In [ ]:
# 2.1.7. Which type of food (rcuisine) is served most among the restaurants?
engine.execute("Select Rcuisine,MAX(cuisineCoint) from (SELECT count(Rcuisine) as cuisineCoint,Rcuisine from (SELECT distinct placeID,Rcuisine FROM restaurants) group by Rcuisine)").fetchall()

In [ ]:
# 2.1.8. What is the percentage of customers who were born between 1980 and 1990?
# len(dfuserprofile[(dfuserprofile.birth_year>=1980) & (dfuserprofile.birth_year<=1990)])/len(dfuserprofile)================================================

# engine.execute('select count((SELECT distinct userID FROM customers where birth_year>=1980 and birth_year<=1990)) as 
# filteredCustomers,count((SELECT distinct userID FROM customers) as totalCustomers))''.fetchall()

# engine.execute('select count((SELECT distinct userID FROM customers where birth_year>=1980 and birth_year<=1990)) as filteredCustomers,count((SELECT distinct userID FROM customers)) as totalCustomers').fetchall()
engine.execute('select (select count(*) from(SELECT distinct userID FROM customers where birth_year>=1980 and birth_year<=1990) as filteredCustomers),(count((SELECT distinct userID FROM customers)))').fetchall()

In [ ]:
# 2.1.9. What is the percentage of students with a medium budget preferring walking to the restaurants?
len(dfuserprofile[(dfuserprofile.budget=='medium') &(dfuserprofile.transport=='on foot')])/len(dfuserprofile)*100
engine.execute("SELECT count(*) FROM customers where transport='on foot' and budget<='medium'").fetchall() #=====================to add to the math


###Assignment2- Part 2:

In [ ]:
# 2.2.1. What are the names of restaurants that do not have public parking lots?
engine.execute("SELECT distinct name FROM restaurants where parking_lot!='public'").fetchall()


In [ ]:
# 2.2.2. What are the addresses of restaurants which only accept ‘cash’?
engine.execute(" SELECT address from (SELECT  distinct placeid,address FROM restaurants where Rpayment='cash') ").fetchall()



In [55]:
# 2.2.3. Name the cities where the restaurants cook and serve ‘fast food’?

engine.execute("SELECT distinct city from restaurants where placeID in (SELECT distinct placeID FROM restaurants where Rcuisine='Fast_Food')").fetchall()



[('victoria',), ('San Luis Potosi',), ('Cuernavaca',), ('?',), ('Soledad',)]

In [69]:
# 2.2.4. What is the most common ‘rating’ among customers with family?
engine.execute("SELECT rating from (Select MAX(ratingcount),rating from (SELECT count(rating) as ratingcount,rating from rating where userID in (SELECT distinct customers.userID  FROM customers where ambience='family') group by rating))").fetchall()


[(2,)]

In [ ]:
# 2.2.5. What types of food (‘rcuisine’) received the highest rank from people with‘low’ budget?
engine.execute("Select distinct Rcuisine,sum(rating) from customers inner join rating on rating.userID=customers.userID where budget='low' group by Rcuisine order by sum(rating)").fetchall()

# engine.execute("SELECT rating,MAX(ratingsum) from (SELECT sum(rating) as ratingsum,rating from rating where userID in (SELECT distinct userID from customers where budget='low') group by rating)").fetchall()
engine.execute("SELECT distinct customers.userID,Rcuisine from customers inner join rating on rating.userID=customers.userID where budget='low'").fetchall()
engine.execute("SELECT distinct userID from customers where budget='low'").fetchall()
engine.execute("SELECT food_rating,placeID as placeID1 from rating where userID in (SELECT distinct userID from customers where budget='low')").fetchall()

engine.execute("SELECT MAX(ratingTotal),Rcuisine from (SELECT Rcuisine,sum(food_rating) as ratingTotal from (SELECT food_rating,Rcuisine from (SELECT food_rating,placeID as placeID1,userID  from rating where userID in (SELECT distinct userID from customers where budget='low')) left join(SELECT distinct placeID,Rcuisine from restaurants ) on placeID=placeID1)group by Rcuisine)").fetchall()
# =================================

In [169]:
# 2.2.6. What is the average of ‘service rating’, received from ‘social drinkers’ about restaurants which just served ‘Wine-Beer’?

# three layer nested select where the firsr select finds avverage
# the middle select finds the customer who are social drinkers and the third restourants which serve alchohal

engine.execute(" SELECT AVG(service_rating) from rating where userID in (SELECT distinct userID from  customers where drink_level='social drinker') and placeID in (SELECT distinct placeID from  restaurants where alcohol='Wine-Beer')").fetchall()



[(1.3863636363636365,)]

In [137]:
# 2.2.7. How many smokers gave zero ‘service rating’ to the restaurants without an open area?

engine.execute("select count(userID) from (SELECT  distinct userID from customers where smoker='true' and userID in (SELECT userID from rating where service_rating=0 and placeID in (SELECT distinct placeID from restaurants where area!='open')))").fetchall()


[(15,)]